In [1]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

# numpy
import numpy

# classifier
from sklearn.linear_model import LogisticRegression

In [2]:
import csv
import string
import re
count = 0
neg_count = 0
pos_count = 0
train_neg = open("train_neg.txt","w")
test_neg = open("test_neg.txt","w")
train_pos = open("train_pos.txt","w")
test_pos = open("test_pos.txt","w")
dict = {}
with open('final.csv', 'rb') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        row[2] = row[2].translate(None, string.punctuation)
        row[2] = re.sub(r'[^\x00-\x7F]+',' ', row[2])
        row[2] = ' '.join(row[2].split())
        row[2] = row[2].lower()
        row[2] = row[2].encode('utf-8').strip()
        
        if row[0] == '0':
            neg_count += 1
            if neg_count > 388:
                test_neg.write(row[2]+"\n")
            else:
                train_neg.write(row[2]+"\n")
        else:
            pos_count += 1
            if pos_count > 422:
                test_pos.write(row[2]+"\n")
            else:
                train_pos.write(row[2]+"\n")
train_pos.close()
train_neg.close()
test_pos.close()
test_neg.close()

In [3]:
docLabels = []
data = []
files = ['train_neg','test_neg','train_pos','test_pos']
count = 0
for file_name in files:
    f = open(file_name + '.txt',"r")
    count = 0
    for line in f.readlines():
        count += 1
        data.append(line.strip())
        docLabels.append(file_name + "-" + str(count))

In [4]:
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
       self.labels_list = labels_list
       self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield LabeledSentence(words=doc.split(),tags=[self.labels_list[idx]])

In [5]:
import gensim
LabeledSentence = gensim.models.doc2vec.LabeledSentence
it = LabeledLineSentence(data, docLabels)
model = gensim.models.Doc2Vec(size=40, window=10, min_count=1, workers=11,dbow_words=1)
model.build_vocab(it)
for epoch in range(30):
    model.train(it)


In [6]:
model.most_similar('good')

[('gift', 0.7016488313674927),
 ('great', 0.6667410135269165),
 ('big', 0.6611874103546143),
 ('docking', 0.6563894748687744),
 ('sexy', 0.6302608251571655),
 ('typical', 0.6160938739776611),
 ('contender', 0.6150690317153931),
 ('known', 0.6143141984939575),
 ('broadcaster', 0.6078404784202576),
 ('billed', 0.5966828465461731)]

In [7]:
model.save('./model.d2v')
model = Doc2Vec.load('./model.d2v')

In [8]:
train_arrays = numpy.zeros((810, 40))
train_labels = numpy.zeros(810)

for i in xrange(388):
    prefix = 'train_neg-' + str(i+1)
    train_arrays[i] = model.docvecs[prefix]
    train_labels[i] = 0

for i in xrange(422):
    prefix = 'train_pos-' + str(i+1)
    train_arrays[388+i] = model.docvecs[prefix]
    train_labels[388+i] = 1


In [9]:
import numpy
test_arrays = numpy.zeros((204, 40))
test_labels = numpy.zeros(204)

for i in xrange(98):
    prefix = 'test_neg-' + str(i+1)
    test_arrays[i] = model.docvecs[prefix]
    test_labels[i] = 0

for i in xrange(106):
    prefix = 'test_pos-' + str(i+1)
    test_arrays[i+98] = model.docvecs[prefix]
    test_labels[i+98] = 1

## Logistic Regression

In [10]:

classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)
classifier.score(test_arrays, test_labels)


0.78431372549019607

## Support Vector Machine

In [11]:
from sklearn import svm
classifier = svm.SVC(kernel='rbf')
classifier.fit(train_arrays, train_labels)
classifier.score(test_arrays, test_labels)

0.71568627450980393

## Random Forest Classifier

In [12]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=25)
classifier.fit(train_arrays, train_labels)
classifier.score(test_arrays, test_labels)

0.84313725490196079